## Importaciones

In [2]:
%matplotlib inline
import pandas as pd
from numpy import NaN
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
path_to_Databases = "C:/Users/andre/OneDrive/Escritorio/Universidad/DM/codigoProyecto/"
title_complete_csv = path_to_Databases+"title_complete_csv.csv" 
title_complete = pd.read_csv(title_complete_csv)

In [3]:
path_to_Databases = "C:/Users/andre/OneDrive/Escritorio/Universidad/DM/codigoProyecto/"
principals_complete_csv = path_to_Databases+"principals_complete_csv.csv" 
principals_complete = pd.read_csv(principals_complete_csv)

In [ ]:
title_complete

In [ ]:
principals_complete

## parte del procesamiento

In [ ]:
print(title_complete.dtypes)

In [52]:
#correccion de tipo de datos
#se cambia \\N por nullos dentro del dataset 
title_complete['tconst'] = title_complete['tconst'].astype(str)
title_complete['genres'].replace('\\N',NaN,inplace=True)
title_complete['genres'] = title_complete['genres'].astype(str)
title_complete['runtimeMinutes'] = pd.to_numeric(title_complete['runtimeMinutes'],errors='coerce', downcast='integer')

In [4]:
#eliminacion de duplicados
duplicados = title_complete[title_complete.duplicated(subset='tconst', keep='first')]
#Display the duplicate rows
print("Columnas duplicadas")
print(duplicados)

# Remove duplicates based on the 'ID' column and keep the first occurrence
title_complete = title_complete.drop_duplicates(subset='tconst', keep='first')
principals_complete_no_duplicates = principals_complete.drop_duplicates(subset='nconst', keep='first')

Columnas duplicadas


In [54]:
#Nuevas columnas para categoria
title_complete['category 1'] = NaN
title_complete['category 2'] = NaN
title_complete['category 3'] = NaN

In [ ]:
#Vamos a ver cuantos generos tenemos en total,así cómo volverlos numericos en base a su posición en la lista y ordenarlos para facilitar la visualización
title_complete['genres'].replace('',)
count = 0
conjunto_Generos = set()
listas_generos = []
contador = []
for i in range(len(title_complete['genres'])):
    generos = title_complete['genres'].iloc[i]
    generos = generos.split(",")
    listas_generos.append(generos)
    for genero in generos:
        conjunto_Generos.add(genero)

listaGeneros = list(conjunto_Generos)
# Remplazar listas de géneros individuales por sus posiciones en la lista global
for generos in listas_generos:
    for i in range(len(generos)):
        genero = generos[i]
        generos[i] = listaGeneros.index(genero)
        contador.append(listaGeneros.index(genero))


for generos in listas_generos:
    generos.sort()


#Probablemente faltan categorias por la muestra
print(len(conjunto_Generos))
print(conjunto_Generos)
print(listas_generos)

In [ ]:
#Ingreamos nuestras categorias numericas cómo dato individual para las nuevas columnas
for i in range(len(title_complete)):

    for j in range(len(listas_generos[i])):
        title_complete['category ' + str(j+1)].iloc[i] = listas_generos[i][j]+1

#Rellenamos los espacios vacios con -1 = no hay categoria / no hay más categorias
title_complete[['category 1','category 2','category 3']] = title_complete[['category 1','category 2','category 3']]

In [ ]:

print(title_complete[['tconst','category 1','category 2','category 3']])

# Analisis Exploratorio

In [ ]:
title_complete
Q1 = title_complete['runtimeMinutes'].quantile(0.25)
Q3 = title_complete['runtimeMinutes'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to keep only non-outliers
title_complete = title_complete[(title_complete['runtimeMinutes'] >= lower_bound) & (title_complete['runtimeMinutes'] <= upper_bound)]

title_complete[['runtimeMinutes']].boxplot()
# Add a title
plt.title('Boxplot of Selected Columns')
# Show the plot
plt.show()

In [ ]:
title_complete.describe()

In [ ]:
#Contamos cuantas peliculas hay po cada genero
df_inicial = pd.DataFrame(contador, columns=['Conteo'])
df = pd.DataFrame(columns=['Conteo','Valor Porcentual'])
# Utilizar la función 'value_counts' para contar las ocurrencias de cada número y ordenarlos egun el indice
df['Conteo'] = pd.DataFrame(df_inicial['Conteo'].value_counts().sort_index())
# Llenar el valor porcentual de cada genero respeceto al total de peliculas
df['Valor Porcentual'] = df['Conteo'] / len(title_complete)
print(df)

In [ ]:
df

In [ ]:
# Cuantas peliculas de cada genero se hacen por año para detemrinar el más popular.

# Utiliza melt para "desplegar" las tres columnas en una sola columna llamada 'category'
title_complete_melted = pd.melt(title_complete, id_vars=['startYear'], value_vars=['category 1', 'category 2', 'category 3'], value_name='category')

# Ahora, realiza groupby en la columna 'startYear' y 'category' para contar las coincidencias
coincidencias_por_categoria = title_complete_melted.groupby(['startYear', 'category'])['category'].count()

# Encuentra la categoría que más se repite por cada año (índice de la categoría con recuento máximo)
categoria_mas_comun_por_año = coincidencias_por_categoria.groupby('startYear').idxmax()

# Obtiene la categoría más común por año
categorias_mas_comunes = categoria_mas_comun_por_año.apply(lambda x: x[1])

# Obtén el número de veces que se repite la categoría más popular por año
numero_repeticiones_mas_popular = coincidencias_por_categoria.groupby('startYear').max()

# Crea un DataFrame con las categorías más comunes y el número de repeticiones
resultado = pd.DataFrame({'CategoriaMasComun': categorias_mas_comunes, 'NumeroRepeticionesMasPopular': numero_repeticiones_mas_popular})

for i in range(len(resultado['CategoriaMasComun'])):
    resultado['CategoriaMasComun'][i] = list(conjunto_Generos)[int(resultado['CategoriaMasComun'][i])-1]

# Combinar año y género más popular en una sola etiqueta para el eje x
etiquetas_x = resultado.index.astype(str) + ' - ' + resultado['CategoriaMasComun']

# Crear una figura y un eje para el gráfico
fig, ax = plt.subplots(figsize=(20, 8))

# Crear un gráfico de barras para mostrar el número de películas por año y género más popular
plt.bar(etiquetas_x, resultado['NumeroRepeticionesMasPopular'])

# Personalizar el gráfico
plt.xlabel('Género Más Popular por año')
plt.ylabel('Número de Películas')
plt.title('Número de Películas por Año y Género Más Popular')
plt.xticks(rotation=90, ha='right')  # Rotar las etiquetas del eje x para mayor legibilidad

# Mostrar el gráfico
plt.tight_layout()
plt.show()



In [ ]:
# Crear una figura y un eje para el gráfico
fig, ax = plt.subplots(figsize=(20, 8))

# Usar Matplotlib para crear un gráfico de barras
title_complete.groupby(['startYear']).size().plot(kind='bar', ax=ax)

# Personalizar el gráfico
ax.set_xlabel('Año')
ax.set_ylabel('Numero de Peliculas')
ax.set_title('Numero de peliculas por año')
plt.xticks(rotation=45)  # Rotar las etiquetas del eje x para mayor legibilidad

# Mostrar el gráfico
plt.tight_layout()
plt.show()

In [ ]:
# Crear una gráfica de barras para los conteos
plt.figure(figsize=(13, 9))  # Ajusta el tamaño de la figura
plt.bar(list(conjunto_Generos), df['Conteo'])
plt.xlabel('Genero')
plt.ylabel('Conteo')
plt.title('Gráfico de Barras de Generos y su Conteo')
plt.xticks(list(conjunto_Generos), list(conjunto_Generos), rotation=90)
plt.show()

In [ ]:
# Crear una gráfica de barras para los conteos
plt.figure(figsize=(13, 9))  # Ajusta el tamaño de la figura
plt.bar(list(conjunto_Generos), df['Valor Porcentual'])
plt.xlabel('Genero')
plt.ylabel('Conteo')
plt.title('Gráfico de Barras de Generos y su Valor Porcentual')
plt.xticks(list(conjunto_Generos), list(conjunto_Generos), rotation=90)
plt.show()

In [ ]:
y = title_complete['numVotes']
x = title_complete['averageRating']

plt.scatter(x, y, c='blue', marker='o', label='Puntos')
plt.xlabel('Eje X')
plt.ylabel('Eje Y')
plt.title('Gráfico de dispersión de ejemplo')
plt.legend()

plt.show()

In [ ]:
y = title_complete['runtimeMinutes']
x = title_complete['numVotes']

plt.scatter(x, y, c='blue', marker='o', label='Puntos')
plt.xlabel('Eje X')
plt.ylabel('Eje Y')
plt.title('Gráfico de dispersión de ejemplo')
plt.legend()

plt.show()

In [ ]:
hist = title_complete['numVotes'].hist()

In [ ]:
hist = title_complete['averageRating'].hist()

In [ ]:
hist = title_complete['runtimeMinutes'].hist() 

In [ ]:
title_complete['numVotes'].plot.kde()

In [ ]:
ax = title_complete['averageRating'].plot.kde() #Ta rara, revisar

In [81]:
#normalizacion de dato
def normalizacionMinMax(column):
    min_val = column.min()
    max_val = column.max()
    normalized_column = (column - min_val) / (max_val - min_val)
    return normalized_column
title_complete['runtimeMinutes'] = normalizacionMinMax(title_complete['runtimeMinutes'])
title_complete['averageRating'] = normalizacionMinMax(title_complete['averageRating'])
title_complete['numVotes'] = normalizacionMinMax(title_complete['numVotes'])

In [ ]:
#creacion de diagrama de cajas para deteccion de autlayers 
title_complete[['numVotes','averageRating','runtimeMinutes']].boxplot()
# Add a title
plt.title('Boxplot of Selected Columns')
# Show the plot
plt.show()

In [ ]:
conteo_true = principals_complete_no_duplicates['isDead'].sum()
conteo_false = len(principals_complete_no_duplicates['isDead']) - conteo_true
etiquetas = ['True', 'False']
conteos = [conteo_true, conteo_false]
plt.bar(etiquetas, conteos)
plt.title('Histograma de Booleanos')
plt.xlabel('Valores')
plt.ylabel('Frecuencia')
plt.show()